In [12]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
import bqplot as bq

In [13]:
#read clean data and create dataframe
df = pd.read_csv("eda_final.csv")
df

,Unnamed: 0,age,gender,height,weight,systolic,diastolic,cholesterol,gluc,smoke,alco,active,cardio,bmi,HTN_stage,age_range,bmi_range
0,0,50.391781,2,168,62.0,110.0,80.0,1,1,0,0,1,0,21.967120,HBP_stage1,50.0,Healthy
1,1,55.419178,1,156,85.0,140.0,90.0,3,1,0,0,1,1,34.927679,HBP_stage2,50.0,Obese
2,2,51.663014,1,165,64.0,130.0,70.0,3,1,0,0,0,1,23.507805,HBP_stage1,50.0,Healthy
3,3,48.282192,2,169,82.0,150.0,100.0,1,1,0,0,1,1,28.710479,HBP_stage2,40.0,Overweight
4,4,47.873973,1,156,56.0,100.0,60.0,1,1,0,0,0,0,23.011177,Normal,40.0,Healthy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69530,69530,52.712329,2,168,76.0,120.0,80.0,1,1,1,0,1,0,26.927438,HBP_stage1,50.0,Overweight
69531,69531,61.920548,1,158,126.0,140.0,90.0,2,2,0,0,1,1,50.472681,HBP_stage2,60.0,High Obese
69532,69532,52.235616,2,183,105.0,180.0,90.0,3,1,0,1,0,1,31.353579,HBP_stage2,50.0,Obese
69533,69533,61.454795,1,163,72.0,135.0,80.0,1,2,0,0,0,1,27.099251,HBP_stage1,60.0,Overweight


In [14]:
#make sure systolic blood pressure is an integer
df["systolic"].astype(int)

0        110
1        140
2        130
3        150
4        100
        ... 
69530    120
69531    140
69532    180
69533    135
69534    120
Name: systolic, Length: 69535, dtype: int64

In [15]:
#create function to categorize blood pressure
def blood_pressure_cat(systolic):
    if systolic >= 180:
        return "Crisis"
    elif (140 <= systolic < 180):
        return "HBP_stage2"
    elif (130 <= systolic < 140):
        return "HBP_stage1"
    elif (120 <= systolic < 130):
        return "Elevated"
    else:
        return "Normal"

In [16]:
#run category function on dataframe
df['HTN_stage'] = df['systolic'].apply(lambda x: blood_pressure_cat(x))

In [17]:
#create function to categorize age
def age_category(age):
    if age < 40:
        return "30-39"
    elif (50 > age >= 40):
        return "40-49"
    elif (60 > age >= 50):
        return "50-59"
    else:
        return "60-69"
#run category function on dataframe
df['age_range_y'] = df['age'].apply(lambda x: age_category(x))

In [18]:
#creating handlers for data attibutes
gender_drop= widgets.Dropdown(options=[("Yes",1),
                                       ("No",0)],description="Include gender:",
                                         style= {'description_width': 'initial'})
run_button = widgets.Button(description="Show statistics",
                           button_style='success')

In [19]:
#function to show first plot
def show_plot_1(b=None):
    gen=gender_drop.value
    plt.figure(figsize= (6,6))
    if gen == 0:
        ax =sns.countplot(y= 'HTN_stage', hue='gender', data= df, 
                          order=["Normal","Elevated","HBP_stage1", "HBP_stage2", "Crisis"])
        plt.title('Systolic Blood Pressure Stages By Gender')
        plt.legend(["female", "male"])
    else:
        ax =sns.countplot(y= 'HTN_stage', data= df, 
                          order=["Normal","Elevated","HBP_stage1", "HBP_stage2", "Crisis"])
        plt.title('Systolic Blood Pressure Stages Overview')   
    plt.ylabel('HTN stage')
    total = len(df['age_range'])
    for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_width()/total)
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        ax.annotate(percentage, (x, y))

#function to show second plot
def show_plot_2(b=None):
    plt.figure(figsize= (6,6))
    sns.countplot(x= 'age_range', hue= 'HTN_stage', data= df, order=["30-39","40-49","50-59","60-69"])
    plt.title('Systolic Blood Pressure Categories For Age Ranges', fontweight= 'bold', fontsize= 12)
    plt.xlabel('Age ranges')
    plt.ylabel('Number of Patients')
    plt.legend(["Normal","Elevated","HBP_stage1", "HBP_stage2", "Crisis"])

#function to show third plot    
def show_plot_3(b=None):
    plt.figure(figsize= (6,6))
    sns.countplot(x= 'HTN_stage', hue= 'cardio', 
                  data= df, order=["Normal","Elevated","HBP_stage1", "HBP_stage2", "Crisis"],
                 palette="Set1")
    plt.title('Systolic Blood Pressure Categories With Cardio Disease Coexistance', fontweight= 'bold', fontsize= 12)
    plt.xlabel('Systolic Blood Pressure Categories')
    plt.ylabel('Number of Patients')
    plt.legend(["cardio disease", "no cardio disease"])

In [20]:
#tabs
@run_button.on_click
def plot_on_click(b):
    out.clear_output(wait=True)
    with out:
        show_plot_1()
        show_plot_2()
        show_plot_3()
        plt.show()

out=widgets.Output()
widget_box=widgets.VBox([gender_drop,run_button])

tab0 = widgets.Text('Age')
tab1 = widgets.Text('BMI')
tab2 = widgets.Text('Blood Pressure')
tab3 = widgets.HBox([widget_box,out])
tab4 = widgets.Text('Diastolic BP')

tab_contents = ['Age', 'BMI', 'Blood Pressure', 'Systolic Blood Pressure', 'Diastolic Blood Pressure']
children = [tab0, tab1, tab2, tab3, tab4]
tab = widgets.Tab()
tab.children = children
for i in range(len(tab_contents)):
    tab.set_title(i, tab_contents[i])
tab